In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('/work/MADS_592/data/hourly-2010.csv')

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (3,33,42,43,46,51,73,95) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
cols = ['HourlyPresentWeatherType','DATE','HourlyPrecipitation','HourlyDryBulbTemperature']
cols_num = [col for col in cols if col != 'HourlyPresentWeatherType']

In [ ]:
df_temp = df.copy()[cols]
df_temp['HourlyPresentWeatherType'].fillna('_',inplace=True)
df_temp['HourlyDryBulbTemperature'].fillna( method='ffill',inplace=True)
df_temp['HourlyPrecipitation']=  df_temp['HourlyPrecipitation'].astype(str)
df_temp['HourlyPrecipitation']=  df_temp['HourlyPrecipitation'].str.strip('s  TS')
df_temp['HourlyPrecipitation'] = pd.to_numeric(df_temp['HourlyPrecipitation'], errors='coerce')
df_temp['HourlyPrecipitation'].fillna( 0,inplace=True)
df_temp['DATE'] = pd.to_datetime(df_temp['DATE'])

In [ ]:
df_temp['rain_dummy_both'] = df_temp['HourlyPresentWeatherType'].str.contains(r'RA').astype(int)

In [ ]:
def ds(x): #daylight savings
    if (x > pd.Timestamp('2010-03-14 02:00:00') ) & (x < pd.Timestamp('2010-10-07 01:00:00') ):
        return x + pd.Timedelta(hours=1)
    else:
        return x


def mornings(x):
    if (x.hour >= 9  ) & (x.hour < 12  ):
        return 1
    else:
        return 0


def afternoons(x):
    if (x.hour >= 12  ) & (x.hour < 15 ):
        return 1
    else:
        return 0

In [ ]:
df_temp['DATE']  =df_temp['DATE'].apply(ds)
df_temp['afternoon_dummy']  = df_temp['DATE'].apply(afternoons)


In [ ]:
mornings_ = (df_temp['DATE'].dt.hour >= 9 ) & (df_temp['DATE'].dt.hour < 12 )
afternoons_ = (df_temp['DATE'].dt.hour >= 12 ) & (df_temp['DATE'].dt.hour < 15 )
df_temp_x = df_temp[mornings_ | afternoons_].reset_index() # drop all data not in morning or afternoon

In [ ]:
def rain_thresh(x, afternoon):
    
    if (x >= 0.05) & (afternoon == 1): 
        retval = 1
    elif (x >= 0.03) & (afternoon == 0): 
        retval = 1
    else:
        retval = 0


    return retval

In [ ]:
df_temp_x['block_rain_total'] = df_temp_x.groupby([df_temp_x['DATE'].dt.date, df_temp_x['afternoon_dummy']])['HourlyPrecipitation'].transform(sum) # cumsum for each block of a day

In [ ]:
df_temp_x['rain_threshold'] = df_temp_x.apply(lambda row: rain_thresh(row['block_rain_total'],row['afternoon_dummy']),axis=1)

In [ ]:
# def make_before_after_rain_dummy(df_gb):         
    
#     # here a dataframe is being passed to the comp_dates function through 'apply'
#     df_gb['before_after_rain_dummy'] = df_gb.apply(
#         lambda row: 0 if row['pickup_datetime_1min'] <= row['b_ts'] else 1, axis=1
#     )   
#     return df_gb

# #df_3wfl = df_3wfl.groupby('b_id').apply(make_before_after_rain_dummy)

In [ ]:
df_temp_x['wet_morning'] = ((df_temp_x['rain_threshold'] ==1) & (df_temp_x['afternoon_dummy'] == 0)).astype(int) # intermediate to get dummy for wet morning

df_temp_x['wet_morning_sum'] = df_temp_x.groupby(df_temp_x['DATE'].dt.date)['wet_morning'].transform(sum) # intermediate to get dummy for wet morning
df_temp_x['wet_morning_today'] = (df_temp_x['wet_morning_sum'] >= 1 ).astype(int) 

In [ ]:
df_temp_x['treatment'] = ((df_temp_x['rain_threshold'] ==1) & (df_temp_x['afternoon_dummy'] ==1) & (df_temp_x['wet_morning_today'] ==0)).astype(int)

In [ ]:
# final_cols = ['DATE','HourlyPresentWeatherType', 'HourlyPrecipitation',\
#        'HourlyDryBulbTemperature',  'afternoon_dummy',\
#        'block_rain_total', 'rain_threshold','wet_morning_today','treatment' ]
# df_final = df_temp_x[final_cols]


In [ ]:
def spec(x): # keep treatment binary in groupby
    if sum(x) > 1:
        return 1
    else:
        return 0

In [ ]:
df_temp_x['HourlyDryBulbTemperature'] = pd.to_numeric(df_temp_x['HourlyDryBulbTemperature'], errors='coerce')

In [ ]:
df_final = df_temp_x.groupby([df_temp_x['DATE'].dt.date, df_temp_x['afternoon_dummy']]).agg({'HourlyPrecipitation':'sum',
'HourlyDryBulbTemperature':'mean','treatment':spec }).reset_index()

In [ ]:
df_final['DATE'] = pd.to_datetime(df_final['DATE'])

def spec2(x): # keep treatment binary in groupby
    if sum(x) == 1:
        return 1
    else:
        return 0

In [ ]:
df_final['treatment'] = df_final.groupby(df_final['DATE'].dt.date)['treatment'].transform(spec2)

In [ ]:
test = df_final[(df_final['afternoon_dummy'] ==0) & (df_final['HourlyPrecipitation'] >= 0.05)]['DATE'].to_list()

In [ ]:
df_final = df_final[~df_final['DATE'].isin(test)] # filter for dates
# df_final.to_csv('/work/MADS_592/data/cb_weather_to_join.csv', index=False ) 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e5c38169-dd81-4fdb-9628-d1699354129b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>